### CMPE 256 Programming Assignment
### Name: Sai Ashrith Aduwala
### SJSU Id: 014427725


Import the necessary libraries.

In [1]:
from surprise import *
import pandas as pd
from surprise import accuracy
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import numpy as np

Read the train and test data. 

In [2]:
df= pd.read_csv('train.csv')         #Load the training data.
tt = pd.read_csv('test.csv')         #Load the testing data.
#tdf = pd.read_csv('newtrain.csv')   #Load the modified training data.
df.head(5)                           #Display the first five rows of the data.

,User,Track,Rating
0,1,0,27
1,4,0,10
2,8,0,30
3,10,0,30
4,11,0,14


Metadata about the training data.

In [3]:
df.describe()

,User,Track,Rating
count,150999.000000,150999.000000,150999.000000
mean,26512.487997,86.650263,36.433274
std,13633.371921,56.020086,22.582832
min,0.000000,0.000000,0.000000
25%,17735.500000,36.000000,15.000000
50%,27878.000000,81.000000,32.000000
75%,35966.000000,142.000000,50.000000
max,50927.000000,183.000000,100.000000


Checking for any null values in the training data.

In [4]:
df.isnull().sum()

User      0
Track     0
Rating    0
dtype: int64

Initializing the reader variable with a scale of 0-100.

Transforming the training data from a dataframe format to a Reader format.

In [5]:
reader = Reader(rating_scale=(0,100))
data=Dataset.load_from_df(df,reader)

Splitting the train data into training set and testing set.

In [9]:
train_df, test_df = train_test_split(data, test_size=0.01)

Creating variables for different algorithms in the surprise library using their default parameters.

In [7]:
blo = BaselineOnly()   #surprise.prediction_algorithms.baseline_only.BaselineOnly(bsl_options={}, verbose=True)

sim_options={
    'user_based':False    #For this problem we need item based model, thus, this value is set to false.
}

knnb = KNNBasic(sim_options=sim_options)   #surprise.prediction_algorithms.knns.KNNBasic(k=40, min_k=1, sim_options={}, verbose=True, **kwargs)

knnm = KNNWithMeans(sim_options=sim_options)   #surprise.prediction_algorithms.knns.KNNWithMeans(k=40, min_k=1, sim_options={}, verbose=True, **kwargs)

knnz = KNNWithZScore(sim_options=sim_options)   #surprise.prediction_algorithms.knns.KNNWithZScore(k=40, min_k=1, sim_options={}, verbose=True, **kwargs)

knnl = KNNBaseline(sim_options=sim_options)   #surprise.prediction_algorithms.knns.KNNBaseline(k=40, min_k=1, sim_options={}, bsl_options={}, verbose=True, **kwargs)

svd = SVD()   #surprise.prediction_algorithms.matrix_factorization.SVD

so = SlopeOne()   #surprise.prediction_algorithms.slope_one.SlopeOne

cc = CoClustering()   #surprise.prediction_algorithms.co_clustering.CoClustering

Building the models with the training set(train_df) and testing them with the test set(test_df).

In [8]:
blo.fit(train_df)                           #BaselineOnly()
pred_blo = blo.test(test_df)

knnb.fit(train_df)                          #KNNBasic()
pred_knnb = knnb.test(test_df)

knnm.fit(train_df)                          #KNNWithMeans()
pred_knnm = knnm.test(test_df)

knnz.fit(train_df)                          #KNNWithZScore()
pred_knnz = knnz.test(test_df)

knnl.fit(train_df)                          #KNNBaseline()
pred_knnl = knnl.test(test_df)

svd.fit(train_df)                           #SVD()
pred_svd = svd.test(test_df)

so.fit(train_df)                            #SlopeOne()
pred_so = so.test(test_df)

cc.fit(train_df)                            #CoClustering()
pred_cc = cc.test(test_df)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


Print out the RMSE scores for each model.

In [9]:
pred_list = [pred_blo, pred_knnb, pred_knnm, pred_knnz, pred_knnl, pred_svd, pred_so, pred_cc]   #list of predictions
model_list = ["BaselineOnly","KNNBasic","KNNWithMeans","KNNWithZScore","KNNBaseline","SVD","SlopeOne","CoClustering"]  #list of algorithms
x=0;
for i in pred_list:
    print(model_list[x])                    #Print the model name.
    x+=1
    accuracy.rmse(i, verbose=True)          #Print the RMSE score for that model.
    print('\n')
    

BaselineOnly
RMSE: 19.6346


KNNBasic
RMSE: 18.8269


KNNWithMeans
RMSE: 17.6878


KNNWithZScore
RMSE: 17.6097


KNNBaseline
RMSE: 17.5664


SVD
RMSE: 16.2095


SlopeOne
RMSE: 18.5940


CoClustering
RMSE: 19.6638




The algorithm with the least RMSE score is SVD. Therefore choose SVD for offline and hyperparameter tuning.

In [ ]:
svd_algo = SVD(n_factors=400, reg_all=0.0001, n_epochs=30)    #Perform offline parameter tuning by manually trying and changing the parameters for the algorithm.
svd_algo.fit(train_df)
prediction = svd_algo.test(test_df)

RMSE score after performing offline parameter tuning.

In [8]:
accuracy.rmse(prediction,verbose=True)

RMSE: 16.5273


16.527291436653663

Performing cross-validation.

In [12]:
svd_cv = SVD(n_factors=300, reg_all = 0.0001)
cross_validate(svd_cv, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    16.8585 16.7417 16.7627 16.7739 16.7961 16.7866 0.0400  
Fit time          19.48   18.95   18.83   19.18   18.86   19.06   0.24    
Test time         0.25    0.25    0.24    0.24    0.26    0.25    0.01    


{'test_rmse': array([16.85845613, 16.74171125, 16.762664  , 16.77386217, 16.79614709]),
 'fit_time': (19.479861736297607,
  18.952356576919556,
  18.828615188598633,
  19.184714555740356,
  18.864559650421143),
 'test_time': (0.2543203830718994,
  0.24733924865722656,
  0.24235105514526367,
  0.2403578758239746,
  0.26427245140075684)}

Performing hyperparameter tuning using GridSearchCV()

In [13]:
parameters = {'n_factors': [300, 400],           #The set of parameters used for model tuning.
              'reg_all': [0.0001, 0.001],
              'n_epochs': [20, 30]
              }

In [14]:
grid = GridSearchCV(SVD, parameters, measures=['rmse'], cv=5)  #Initialize the GridSearchCV() 

In [15]:
grid.fit(data)   #Build and test the model using the different combinations of the above given parameters.

In [16]:
print(grid.best_score['rmse'])    #Print the best RMSE score achieved.

16.77208423326956


In [17]:
print(grid.best_params['rmse'])   #Print the parameters for which we got the best RMSE score.

{'n_factors': 300, 'reg_all': 0.0001, 'n_epochs': 30}


In [18]:
svd_grid = grid.best_estimator['rmse']   #Using the best parameters rebuild the model using the full trainset.
svd_grid.fit(data.build_full_trainset())    

Predict the rating for the users in the test data read from the 'test.csv' file.

In [19]:
file = open('predictions.csv', 'w')         #Create/open a file to store the predictions
header="Id,Rating"                          #Define the headers for the prediction file
file.write(header+"\n")                     #Insert the headers to the file
for i in range(0, len(tt)):
    predicted_rating = 0.0
    output = None
    pred=svd_grid.predict(uid=tt.iloc[i]['User'],iid=tt.iloc[i]['Track'])        #Predict the rating for each user in the test data by giving the 'User' and 'Track' cloumns as the parameters.
    output =str(pred[0]) + "-" + str(pred[1]) + ","                          #Format the 'User' and 'Track' numbers and store in 'output' variable.
    predicted_rating = predicted_rating + float(pred[3])                     #Update the predicted rating for the above user and track.
    output = output + str(predicted_rating)                                  #Store the predicted rating in the 'output' variable.
    file.write(output + "\n")                                                #Write the value of the 'output' variable to the 'predictions.csv' file.
    
file.close()                                #Close the file.